决策树分析：预测什么样的人可能是独生子

In [ ]:
set.seed(42)
library(tidyverse)
library(rpart.plot)
library(rsample)

In [ ]:
getwd()

In [ ]:
YPS_file_path <- getwd() |> dirname() |> paste("/data/YoungPeopleSurvey/responses.csv",sep="") |> file.path()
YPS_file_path
YPS_dt <- read.csv(YPS_file_path)
dim(YPS_dt)
head(YPS_dt)

查看数据类型

In [18]:
glimpse(YPS_dt)

Rows: 686
Columns: 150
$ Music                          <int> 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ~
$ Slow.songs.or.fast.songs       <int> 3, 4, 5, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, ~
$ Dance                          <int> 2, 2, 2, 4, 2, 5, 3, 2, 3, 1, 1, 5, 2, ~
$ Folk                           <int> 1, 1, 2, 3, 3, 3, 2, 5, 2, 1, 2, 3, 1, ~
$ Country                        <int> 2, 1, 3, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, ~
$ Classical.music                <int> 2, 1, 4, 4, 3, 2, 2, 2, 2, 4, 4, 1, 2, ~
$ Musical                        <int> 1, 2, 5, 3, 3, 2, 2, 5, 3, 1, 3, 5, 3, ~
$ Pop                            <int> 5, 3, 3, 5, 2, 5, 4, 3, 4, 2, 3, 5, 4, ~
$ Rock                           <int> 5, 5, 5, 3, 5, 3, 5, 5, 3, 5, 5, 2, 5, ~
$ Metal.or.Hardrock              <int> 1, 4, 3, 1, 5, 1, 1, 2, 2, 1, 4, 1, 2, ~
$ Punk                           <int> 1, 4, 4, 2, 3, 1, 2, 3, 1, 1, 2, 1, 5, ~
$ Hiphop..Rap                    <int> 1, 1, 1, 5, 4, 3, 3, 2, 3, 1, 3, 2, 3, ~
$ Reggae..Ska    

先看看NA的情况

In [4]:
YPS_dt %>% summary()

     Music       Slow.songs.or.fast.songs     Dance            Folk      
 Min.   :1.000   Min.   :1.000            Min.   :1.000   Min.   :1.000  
 1st Qu.:5.000   1st Qu.:3.000            1st Qu.:2.000   1st Qu.:1.000  
 Median :5.000   Median :3.000            Median :3.000   Median :2.000  
 Mean   :4.732   Mean   :3.328            Mean   :3.113   Mean   :2.289  
 3rd Qu.:5.000   3rd Qu.:4.000            3rd Qu.:4.000   3rd Qu.:3.000  
 Max.   :5.000   Max.   :5.000            Max.   :5.000   Max.   :5.000  
 NA's   :3       NA's   :2                NA's   :4       NA's   :5      
    Country      Classical.music    Musical           Pop       
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.000  
 1st Qu.:1.000   1st Qu.:2.000   1st Qu.:2.000   1st Qu.:3.000  
 Median :2.000   Median :3.000   Median :3.000   Median :4.000  
 Mean   :2.123   Mean   :2.956   Mean   :2.762   Mean   :3.472  
 3rd Qu.:3.000   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.000  
 Max.   :5.000   M

清洗掉有NA的数据

In [15]:
YPS_dt <- na.omit(YPS_dt)
YPS_dt %>% summary()

     Music       Slow.songs.or.fast.songs     Dance           Folk      
 Min.   :1.000   Min.   :1.000            Min.   :1.00   Min.   :1.000  
 1st Qu.:5.000   1st Qu.:3.000            1st Qu.:2.00   1st Qu.:1.000  
 Median :5.000   Median :3.000            Median :3.00   Median :2.000  
 Mean   :4.759   Mean   :3.294            Mean   :3.07   Mean   :2.258  
 3rd Qu.:5.000   3rd Qu.:4.000            3rd Qu.:4.00   3rd Qu.:3.000  
 Max.   :5.000   Max.   :5.000            Max.   :5.00   Max.   :5.000  
    Country      Classical.music    Musical           Pop            Rock      
 Min.   :1.000   Min.   :1.000   Min.   :1.000   Min.   :1.00   Min.   :1.000  
 1st Qu.:1.000   1st Qu.:2.000   1st Qu.:2.000   1st Qu.:3.00   1st Qu.:3.000  
 Median :2.000   Median :3.000   Median :3.000   Median :4.00   Median :4.000  
 Mean   :2.112   Mean   :2.981   Mean   :2.759   Mean   :3.44   Mean   :3.787  
 3rd Qu.:3.000   3rd Qu.:4.000   3rd Qu.:4.000   3rd Qu.:4.00   3rd Qu.:5.000  
 Max.   :

看看大概分别有多少样本，感觉还行，，，

In [16]:
YPS_dt %>%
    group_by(Only.child) %>%
    summarize(n=n())

Only.child,n
<chr>,<int>
,1
no,526
yes,159


In [17]:
unique(YPS_dt[,"Only.child"])

[1] "no"  "yes" ""

将数据集分为训练集和测试集

In [11]:
YPS_split <- initial_split(YPS_dt,strata = Only.child,prop=0.75)
YPS_training <- training(YPS_split)
YPS_testing <- testing(YPS_split)

通过比例确定数据集的层化抽样是成功的

In [12]:
YPS_training %>%
    group_by(Only.child) %>%
    summarize(n=n(),prop=n()/nrow(YPS_training))

YPS_testing %>%
    group_by(Only.child) %>%
    summarize(n=n(),prop=n()/nrow(YPS_testing))

Only.child,n,prop
<chr>,<int>,<dbl>
,1,0.001945525
no,394,0.766536965
yes,119,0.231517510


Only.child,n,prop
<chr>,<int>,<dbl>
no,132,0.7674419
yes,40,0.2325581
